In [163]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from Bio import Seq, SeqIO, SeqRecord, Alphabet
from Bio.SeqFeature import SeqFeature, FeatureLocation, BeforePosition

In [164]:
with open('./cds.fasta') as f:
    cds = list(SeqIO.parse(f, format='fasta', alphabet=Alphabet.generic_dna))
    
with open('./regulatory.fasta') as f:
    regulatory = list(SeqIO.parse(f, format='fasta', alphabet=Alphabet.generic_dna))
    
with open('./other-reporters.gb') as f:
    reporters = list(SeqIO.parse(f, format='genbank'))

In [165]:
reporters

[SeqRecord(seq=Seq('ATGTCTTATTCAAAGCATGGCATCGTACAAGAAATGAAGACGAAATACCATATG...TAA', IUPACAmbiguousDNA()), id='amilGFP_(yellow)', name='amilGFP_(yellow)', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGCTTCACTGGTTAAAAAAGACATGTGCATCAAAATGACGATGGAAGGAACA...TAA', IUPACAmbiguousDNA()), id='aeBlue', name='aeBlue', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGCTTCCAAAATAAGCGACAACGTACGTATCAAACTGTATATGGAGGGCACG...TAA', IUPACAmbiguousDNA()), id='cjBlue', name='cjBlue', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGTCGGTAATCAAGCAAGTGATGAAAACTAAGCTTCATCTTGAAGGTACCGTT...TAA', IUPACAmbiguousDNA()), id='eforRed_(pink)', name='eforRed_(pink)', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGCTTCCTCCGAAGACGTTATCAAAGAGTTCATGCGTTTCAAAGTTCGTATG...TAA', IUPACAmbiguousDNA()), id='RFP_(red)', name='RFP_(red)', description='', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGCGAGCTTCCTGAAAAAGACGATGCCGTTCAAAACCACGATTGAAGGCACG...TAA', IUPACAmbiguousDNA()), id='asPink', name='asPink', description='',

In [166]:
for part in cds:
    part.features.append(SeqFeature(FeatureLocation(0,len(part)), type="CDS", qualifiers={'label': part.name, 'gene': part.name}))
    
for part in regulatory:
    r = "regulatory"
    c = "misc_signal"
    if part.name[0] == 'P':
        c = "promoter"
    elif part.name[0] == 'U':
        r = "5'UTR"
        c = 'UTR'
    elif part.name[0] == "T":
        c = 'terminator'
    
    feat = SeqFeature(FeatureLocation(0,len(part)), type=r, qualifiers={'standard_name': part.name, 'label': part.name})
    if r == 'regulatory': feat.qualifiers['regulatory_class'] = c
    part.features.append(feat)

In [167]:
moclo = {
    'X': 'GGAG',
    'i0': 'TACT',
    'i1': 'A', # We expect an ATG after this
    'i2': 'GCTT',
    'Y': 'CGCT'
}

for name, seq in moclo.items():
    seqr = SeqRecord.SeqRecord(Seq.Seq(seq, alphabet=Alphabet.generic_dna), id=name, name=name)
    seqr.features.append(SeqFeature(FeatureLocation(0, BeforePosition(4)), type="misc_feature", qualifiers={'function': 'MoClo Adapter', 'standard_name': name}))
    moclo[name] = seqr
    
bsaI_forward = SeqRecord.SeqRecord(Seq.Seq('GGTCTCA', alphabet=Alphabet.generic_dna))
bsaI_forward.features.append(SeqFeature(FeatureLocation(0, len(bsaI_forward)), type="misc_feature", qualifiers={'function': 'MoClo RE Site', 'standard_name': 'BsaI'}))

In [168]:
parts = []

for part in regulatory + cds + reporters:
    if part.features[0].type.upper() == 'CDS':
        part_fragment = bsaI_forward + moclo['i1'] + part + moclo['i2'] + bsaI_forward.reverse_complement()
        part_fragment
    elif part.name[0] == 'P': # promoter
        part_fragment = bsaI_forward + moclo['X'] + part + moclo['i0'] + bsaI_forward.reverse_complement()
    elif part.name[0] == 'T': # terminator
        part_fragment = bsaI_forward + moclo['i2'] + part + moclo['Y'] + bsaI_forward.reverse_complement()
    elif part.name[0:3] == 'UTR': # utr
        part_fragment = bsaI_forward + moclo['i0'] + part + moclo['i1'] + bsaI_forward.reverse_complement()
        
    part_fragment.id = part.id
    part_fragment.name = part.name
    part_fragment.description = "MoClo assembly fragment for part {}".format(part.name)
    parts.append(part_fragment)

In [169]:
SeqIO.write(parts, "parts.gb", "genbank")

60